In [63]:
from sframe import SFrame
from IPython.core.display import display

In [64]:
import pandas as pd
import json
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from multiprocessing import Process, Pool
from threading import Thread
import time
import timeit
import numpy as np
%matplotlib inline

# Nacteni dat

In [67]:
ndataset = "goout"

In [68]:
with open("/home/kuba/ownCloud/ModGen-fac-mat/sample-data-v2/"+ndataset+"/items.json",'r') as f:
    items = json.loads(f.read())

with open("/home/kuba/ownCloud/ModGen-fac-mat/sample-data-v2/"+ndataset+"/properties.json",'r') as f:
    properties = json.loads(f.read())
    
with open("/home/kuba/ownCloud/ModGen-fac-mat/sample-data-v2/"+ndataset+"/user.folds.json",'r') as f:
    user_folds = json.loads(f.read())
    
with open("/home/kuba/ownCloud/ModGen-fac-mat/sample-data-v2/"+ndataset+"/users.int2str.json",'r') as f:
    users_int2str = json.loads(f.read())
    
with open("/home/kuba/ownCloud/ModGen-fac-mat/sample-data-v2/"+ndataset+"/users.str2int.json",'r') as f:
    users_str2int = json.loads(f.read())
      
ratings = pd.read_csv("/home/kuba/ownCloud/ModGen-fac-mat/sample-data-v2/"+ndataset+"/ratings.csv",  ) #dtype = {'rating': np.float, 'itemId': str, 'userId':str}
# ratings = SFrame.read_csv("/home/kuba/ownCloud/ModGen-fac-mat/sample-data-v2/"+ndataset+"/ratings.csv", column_type_hints=[str,str,float] ) 
with open("/home/kuba/ownCloud/ModGen-fac-mat/sample-data-v2/"+ndataset+"/items.int2str.json",'r') as f:
    items_int2str = json.loads(f.read())
    
with open("/home/kuba/ownCloud/ModGen-fac-mat/sample-data-v2/"+ndataset+"/items.str2int.json",'r') as f:
    items_str2int = json.loads(f.read())

/usr/local/lib/python3.4/dist-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [73]:
np.ones(shape = (len(items_int2str),len(items_int2str)), dtype=float)

MemoryError: 

In [77]:
pd.DataFrame(np.ones(1)).

AttributeError: 'DataFrame' object has no attribute 'memory'

In [74]:
len(items_int2str)*len(items_int2str)

2920970116

# Faktorizace

In [40]:
import multiprocessing
import ctypes
import numpy as np
import scipy.sparse as sp
import scipy.sparse.linalg as sparse_lg
import datetime
from math import sqrt

In [41]:
import platform
platform.architecture()

('64bit', 'ELF')

In [42]:
import numpy as np
from numpy.linalg import inv
# %load_ext line_profiler

In [58]:
class MatrixFactorization:
    def __init__(self, dataset, user_fold, no_factors, relevant = .25, testset_amount = 0.2, scale = True):
        self.user_fold = user_fold
        ratings = dataset[~dataset.userId.isin(user_fold)].copy()
      
        ratings['userId_'] = ratings['userId'].apply(lambda id_: users_str2int[str(id_)])
        ratings['itemId_'] = ratings['itemId'].apply(lambda id_: items_str2int[str(id_)])
        self.Dataset = ratings
        
        
        self.Users = set(ratings['userId_'])
        self.Items = set(ratings['itemId_'])
        
        self.no_Users = len(self.Users)
        self.no_Items = len(self.Items)
        
        self.user_map_dict = dict(zip(self.Users,range(self.no_Users)))
        self.item_map_dist = dict(zip(self.Items,range(self.no_Items)))

        self.u_map = lambda users : map(lambda id_: user_map_dict[id_], users)
        self.i_map = lambda items : map(lambda id_: item_map_dist[id_], items)

        trainset, testset = self.init_testset(self.Dataset.copy(), testset_amount, relevant)
        
        self.Ratings_trainset = trainset
        self.Ratings_testset = testset

        
        # userId -> items ids
        self.User_Items = self.columns_to_dict(self.Ratings_trainset, 'userId_', 'itemId_')
        self.Item_Users = self.columns_to_dict(self.Ratings_trainset, 'itemId_', 'userId_')
        
        # itemId -> ratings
        self.Item_Ratings = self.columns_to_dict(self.Ratings_trainset, 'itemId_', 'rating')
        self.User_Ratings = self.columns_to_dict(self.Ratings_trainset, 'userId_', 'rating')
        
        # (userId_, itemId_) -> rating[]
        self.User_Item_Rating_train = self.ratings_to_dict(self.Ratings_trainset, ['userId_', 'itemId_'], ['rating'])
        self.User_Item_Rating_test = self.ratings_to_dict(self.Ratings_testset, ['userId_', 'itemId_'], ['rating'])
        
        
        self.relevant = relevant
        self.testset_amount = testset_amount
        self.no_factors = no_factors
        
      
    
    def columns_to_dict(self, df, key_column, value_column):
        dict_ = {}
        for tuple_ in df[[key_column, value_column]].itertuples():
            key = tuple_[1]
            value = tuple_[2]
            if(not key in dict_.keys()):
                dict_[key] = []

            dict_[key].append(value)
        return dict_   
    
    def ratings_to_dict(self, df, key_columns, value_column):
        dict_ = {}
        
        for tuple_ in df[key_columns + value_column].itertuples():
            key1 = tuple_[1]
            key2 = tuple_[2]
            value = tuple_[3]
            
            if(not (key1,key2) in dict_.keys()):
                dict_[(key1,key2)] = []

            dict_[(key1,key2)].append(value)
        return dict_   

    def init(self):
        if(self.random_init): 
            user_shared_array_base = multiprocessing.Array(ctypes.c_double, np.random.rand(self.no_Users * self.no_factors), lock=False) #TODO pepople in fold vector ZEROS
            user_shared_array = np.frombuffer(user_shared_array_base,dtype=float)    
            self.Users = user_shared_array.reshape(self.no_Users, self.no_factors)
            
            item_shared_array_base = multiprocessing.Array(ctypes.c_double, np.random.rand(self.no_Items * self.no_factors), lock= False)
            item_shared_array = np.frombuffer(item_shared_array_base,dtype=float)
            self.Items = item_shared_array.reshape(self.no_Items, self.no_factors)
            
    def init_optimizer(self):
        if(self.weights_mode == "AllRank"):
            print("** Set weight: ",self.weight, " to missing ratings **")

        elif(self.weights_mode == "AllRank-pop"):
            print("AllRank-pop")
        
        elif(self.weights_mode == "MF-RMSE"):
            assert self.weight == 0, "Weight of missiong values MF-RMSE mode must be 0!"
            
        if(self.imputation_value != 0 ):   
            print("** Surrogate missing rating values by imputation value: ", self.imputation_value, " **")
            
        
    def init_testset(self, ratings, testset_amount, relevant):
        ratings["Testset"] = False
        relevant_ = ratings.loc[ratings['rating']>=relevant]
        
        for user in self.Users:
            ratings_ = relevant_.loc[relevant_['userId_']==user]
            if(not ratings_.empty):
                indeces = ratings_.index.tolist()
                sample_indeces = np.random.choice(indeces, int(np.ceil(len(indeces)*relevant)), replace =False)
                ratings.loc[sample_indeces, "Testset"]=True
                
        return ratings[ratings.Testset==False], ratings[ratings.Testset==True]

            
            
        
    '''
    ATOP
    '''
    
    def NRANKs_u(self, user, items):
        prediction = self.R_hat[:,user]
        ranks = np.argsort(prediction)
        N = len(prediction)
        nranks = np.array([np.where(ranks==item)[0][0] for item in items])/N
        return nranks

   
    
    def ATOP(self):
        atop = []
        RMSE = []
        for user, cont_id_rat in self.Testset.items():
            RMSE.extend(self.R_hat[cont_id_rat['indices'], user] - cont_id_rat['ratings'])
            nranks = self.NRANKs_u(user, cont_id_rat['indices'])
            atop.extend(nranks)
        
        return np.mean(atop), sqrt(np.mean((np.array(RMSE)**2)))
            
    
    def RECALL_u(self, u, K, beta):
        prediction = self.R_hat[:,u]
        T = self.Test_matrix
        idx = np.argsort(prediction)[::-1][:K]
        idx_relevant = np.where(T[:,u] == True)
    
        return np.sum(T[idx,u])/np.sum(T[:,u]), np.sum([1/(self.number_of_obs_relevant_items[i]**beta) for i in idx_relevant[0]]) #TODO IndexError: index 7591 is out of bounds for axis 0 with size 717


    
    def RECALL(self, K, beta):
        recall = 0
        for u in range(len(self.Users)):
            r, w = self.RECALL_u(u, K, beta)
            recall += r*w
        return recall
             
        
    '''
    RMSE
    '''  
    def RMSE(self, ratings):
        users = self.Users
        items = self.Items
        
        errors = []
        for tuples_ in ratings[['userId_','itemId_','rating']].itertuples():
            user, item, rating = tuples_[1], tuples_[2], tuples_[3] 
            errors.append(rating - (self.imputation_value + np.sum(np.array(V[item])*np.array(U[user]))))

        return sqrt(((np.array(errors)**2).sum())/len(errors))
    
    '''
    OPTIMAZE PER FACTOR
    '''
    def item_factor_factor(self,batch):
        V = self.Weights, self.Ratings_matrix, self.Items
        U = sp.csr_matrix(self.Users)
        r_m, lambda_ = self.imputation_value, self.lambda_

        UU = (self.weight*U.T).dot(U)
        for i in batch:
            i_rated = self.Item_Users[i]
            U_s = U[self.u_map(i_rated),:]
            Wi = np.empty(shape=(1,len(self.Item_Users[i])))
            Wi.fill(self.weight_of_rating)
            
            lM = sp.csr_matrix(np.array(self.Item_Ratings[i]) - r_m).dot( sp.csr_matrix( Wi.T.dot(np.ones((1,self.no_factors)))).multiply(U_s) )
            rM = UU - (self.weight*U_s.T).dot(U_s) + U_s.T.multiply(sp.csr_matrix(np.ones((self.no_factors,1)).dot(Wi))).dot(U_s) 
            reg = lambda_ * (self.weight * (self.no_Items-len(Wi)) + (Wi-self.weight).sum()) * sp.eye(self.no_factors)
            V[self.i_map([i]),:] = sparse_lg.spsolve(rM+reg,lM.T)

    
    def compute_user_factor(self, batch):
        U = self.Users
        lambda_, r_m = self.lambda_, self.imputation_value
        V = sp.csr_matrix(self.Items)
        
        VV = (self.weight*V.T).dot(V)
        for u in batch:
            u_rated = self.User_Items[u]
            V_s = V[self.i_map(u_rated),:]
            Wu = np.empty(shape=(len(self.User_Items[u]), u)).fill(self.weight_of_rating)
            Wu.fill(self.weight_of_rating)
          
            lM = sp.csr_matrix(np.array(self.User_ratings[u]) - r_m).dot(sp.csr_matrix( Wu.T.dot(np.ones((self.no_factors,1)))).multiply(V_s) )
            rM = VV - (self.weight*V_s.T).dot(V_s) + V_s.T.multiply(sp.csr_matrix(np.ones((self.no_factors,1)).dot(Wu))).dot(V_s) 
            reg = lambda_ * W[:,u].sum() * (self.weight * (self.no_Users-len(Wu)) + (Wu-self.weight).sum()) * sp.eye(self.no_factors)
                                                                    
            U[self.u_map([u]),:] = parse_lg.spsolve(rM+reg,lM.T)
           

    '''
    OPTIMIZE RMSE
    '''
    def optimize_rmse(self):
        self.init()
        self.init_optimizer()
        
        weighted_errors = []
        ATOPs = []
        
        step_item = int(np.ceil(len(self.Items)/float(self.no_process)))
        step_user = int(np.ceil(len(self.Users)/float(self.no_process)))
        
        item_range = [self.Items[i,i+step_item] for i in range(0, len(self.Items), step_item)]
        user_range = [self.Users[u,u+step_user] for u in range(0, len(self.Users), step_user)]
                                                                    
        print("Item range", item_range)
        print("User range", user_range)
        print("*******************************")
        print("Lambda: ", self.lambda_)
        print("Impute value: ", self.imputation_value)
        print("Weight: ", self.weight)
        print("*******************************")
        
        for ii in range(self.no_iterations):
            if(not ii % 1):
                weighted_errors.append(self.RMSE(self.ratings_trainset))
                print('{}th iteration is completed'.format(ii))
                print("RMSE (TRAIN) ", weighted_errors[-1])
                
#                 atop, rmse = self.ATOP()
#                 ATOPs.append(atop)
#                 print("ATOP ", atop, " RMSE (TEST) ", rmse)

            #ITEMS                
            process = []
            for batch in item_range:
                p = Process(target = self.compute_item_factor, args = (batch,))
                p.daemon = True

                process.append(p)
                p.start()
            
            [p.join() for p in process]
                
            #USERS
            process = []
            for batch in user_range:
                p = Process(target = self.compute_user_factor, args = (batch,))
                p.daemon = True

                process.append(p)
                p.start()
            
            [p.join() for p in process]

        return weighted_errors, ATOPs

        
    def optimaze(self, no_iterations, loss_function="RMSE", opt_method="solver",  lambda_ = 0.001, alfa=0.00001, 
                 no_process = 1, random_init = True, weights_mode = "MF-RMSE", weight = 0, imputation_value = 0, sparse = False, beta = 0, K = 10, weight_of_rating = 1):
        
        self.loss_function = loss_function
        self.solve_opt_method = opt_method 
        self.lambda_ = lambda_
        self.alfa = alfa
        self.no_process = no_process
        self.no_iterations = no_iterations
        self.random_init = random_init
        self.weights_mode = weights_mode
        self.weight = weight
        self.imputation_value = imputation_value
        self.opt_method = opt_method
        self.sparse = sparse
        self.beta = beta 
        self.K = K
        self.weight_of_rating = weight_of_rating
                                                                    
        
        weighted_errors, ATOPs = self.optimize_rmse()
        
        self.plot_rmse(weighted_errors)
        return np.max(ATOPs)
    '''
    PLOT AND EXPLORE
    '''
    def plot_rmse(self, weighted_errors):
        plt.plot(np.log(weighted_errors), label="weighted error: "+str(weighted_errors[-1]))
        plt.ylabel("RMSE log scale")
        plt.xlabel("no iterations")

        plt.legend()
        plt.show()
        
    def explore(self):
        relevant = self.relevant 
        R = self.Ratings_matrix
        no_ratings = np.sum(self.Idx==True)
        no_missing = np.sum(self.Idx==False)
        no_all = no_ratings + no_missing
        
        sizes = [no_ratings, no_missing]
        labels = ["ratings", "missings"]
        colors = ['yellowgreen', 'lightskyblue']

        plt.pie(sizes, labels=labels,
                autopct='%1.1f%%', shadow=True, startangle=90, colors= colors)
        plt.axis('equal')
        plt.show()
        print("#{} rating, #{} missing".format(no_ratings, no_missing))
        print("Estimate offset w(m): ", no_ratings/no_missing)
        print("Avg of ratings: ", R[self.Idx].mean())
        
        no_relevant = np.sum(R[self.Idx]>=relevant)
        no_irelevant = np.sum(R[self.Idx]<relevant)
        labels = ["relevant", "irelevenat"]

        sizes = [no_relevant, no_irelevant]
        colors = ['lightskyblue', 'lightcoral']
        plt.pie(sizes, labels=labels,
                autopct='%1.1f%%', shadow=True, startangle=90, colors= colors)
        plt.axis('equal')
        plt.show()
        
        print("#{} relevant, #{} irelevant".format(no_relevant, no_irelevant))

        items_popularity = np.sort(list(self.number_of_obs_relevant_items.values()))[::-1]
#         plt.bar(range(len(items_popularity)),items_popularity)
        plt.xticks([])
        plt.ylabel("# of item rating marked as relevant")
        plt.xlabel("item")
        plt.plot(items_popularity,'_')
        plt.show()
        
        plt.hist(self.Dataset['rating'].values,bins = len(set(self.Dataset['rating'])))
        plt.xlabel("rating")
        plt.show()

In [59]:
# %load_ext line_profiler

In [60]:
ifold = 0
MFact = MatrixFactorization(ratings.copy(), user_folds[ifold],  50, relevant = 1., testset_amount = 0.2)


In [ ]:
# MFact.explore()

In [26]:
ATOP = MFact.optimaze(6, opt_method="solver",  lambda_ = 0.05, no_process = 7, weights_mode = "AllRank", weight = 0.05, imputation_value = -0.5 , random_init = True, sparse = True,)


** Set weight:  0.05  to missing ratings **
** Surrogate missing rating values by imputation value:  -0.5  **


IndexError: index 529 is out of bounds for axis 1 with size 50

In [ ]:
ATOPs = {}
for lambda_ in [0.01, 0.02, 0.03, 0.04, 0.05, 0.06]:
    ATOPs[lambda_] = {}
    for impute in [-0.5, 0, 0.5]:
        ATOPs[lambda_][impute] = []
        for weight in [0, 0.02, 0.05, 0.08,  0.1, 0.3, 0.5, 0.8, 1]:
            ATOP = MFact.optimaze(6, opt_method="solver",  lambda_ = lambda_, no_process = 7, weights_mode = "AllRank", weight = weight, scale = True, imputation_value = impute , random_init = True, sparse = True, beta= 0.0, K = 20)
            if(ATOPs[lambda_][impute]==[]):
                 ATOPs[lambda_][impute].append(ATOP)
            else:
                if(ATOPs[lambda_][impute][-1]>ATOP):
                    ATOPs[lambda_][impute].append(ATOP)
                    break
                else:
                    ATOPs[lambda_][impute].append(ATOP)
# start_time = datetime.datetime.now()
# datetime.datetime.now() - start_time

In [ ]:
plt.bar(range(len(ATOPs)),ATOPs)
plt.xticks(np.arange(len(ATOPs))+0.5, [0, 0.02, 0.05, 0.08,  0.1, 0.3, 0.5, 0.8, 1])
plt.show()

In [ ]:
for key in ATOPs.keys():
    print("lambda: ", key)
    for im in ATOPs[key].keys():
        print("     impute value: ", im)
        print("                  ", ATOPs[key][im])

In [76]:
W, R= MFact.Weights, MFact.Ratings_matrix
V = MFact.Items[:R.shape[0]]
U = sp.csr_matrix(MFact.Users[:R.shape[1]])
r_m = MFact.imputation_value
lambda_ = MFact.lambda_
UU = (MFact.weight*U.T).dot(U)

i = 0
i_rated = MFact.Idx[i,:]
U_s = U[i_rated,:]
lM = sp.csr_matrix(R[i,:] - r_m).dot(sp.diags(W[i], 0)).dot(U)
rM = UU - (MFact.weight*U_s.T).dot(U_s) + U_s.T.multiply(sp.csr_matrix(np.ones((MFact.no_factors,1)).dot(MFact.Weights[i,i_rated].reshape(1, MFact.Weights[i,i_rated].shape[0])))).dot(U_s) 
reg = lambda_ * W[i].sum() * sp.eye(MFact.no_factors)
sparse_lg.spsolve(rM+reg,lM.T)


array([ 0.03922827,  0.03625654,  0.01451074,  0.01327572, -0.07271202,
       -0.0629774 , -0.0436064 , -0.01009882, -0.05452575,  0.00619541,
       -0.04376823, -0.00383725, -0.02075124, -0.00304385,  0.01887179,
       -0.03466216, -0.00503142,  0.07140535, -0.03133925,  0.05285002,
        0.00261563,  0.0780847 ,  0.03354193, -0.000628  , -0.00158779,
       -0.03234673,  0.01461181,  0.00217666,  0.00316245,  0.06487512,
        0.01479268,  0.00116855, -0.01768169, -0.03174568, -0.01962428,
       -0.01409619,  0.0083385 ,  0.07628607,  0.0257955 ,  0.01359626,
        0.04752589,  0.0331736 , -0.04989023, -0.01649847, -0.04114553,
        0.02133173,  0.01789828, -0.02593577,  0.05394011,  0.00844431])

## optimalizovane 

In [77]:
W, R= MFact.Weights, MFact.Ratings_matrix
V = MFact.Items[:R.shape[0]]
U = sp.csr_matrix(MFact.Users[:R.shape[1]])
r_m = MFact.imputation_value
lambda_ = MFact.lambda_
UU = (MFact.weight*U.T).dot(U)

i_rated = MFact.Idx[i,:]
U_s = U[i_rated,:]
lM = sp.csr_matrix(R[i,i_rated] - r_m).dot( sp.csr_matrix( W[i, i_rated].reshape((1, W[i, i_rated].shape[0])).T.dot(np.ones((1,MFact.no_factors)))).multiply(U[i_rated,:]) )
rM = UU - (MFact.weight*U_s.T).dot(U_s) + U_s.T.multiply(sp.csr_matrix(np.ones((MFact.no_factors,1)).dot(MFact.Weights[i,i_rated].reshape(1, MFact.Weights[i,i_rated].shape[0])))).dot(U_s) 
reg = lambda_ * (MFact.weight * (len(W[i,:])-len(W[i,i_rated])) + (W[i,i_rated]-MFact.weight).sum()) * sp.eye(MFact.no_factors)
sparse_lg.spsolve(rM+reg,lM.T)


array([ 0.03942559,  0.03642063,  0.01456956,  0.01332304, -0.07310275,
       -0.06332515, -0.04385188, -0.0101677 , -0.05480824,  0.00621141,
       -0.04396447, -0.00388706, -0.02090028, -0.00309698,  0.01894411,
       -0.03487651, -0.00507792,  0.0717641 , -0.03152139,  0.05313088,
        0.00260641,  0.0784675 ,  0.03372638, -0.00065793, -0.00161696,
       -0.03253757,  0.0146964 ,  0.00215189,  0.00315795,  0.06519347,
        0.01485466,  0.00116868, -0.01777106, -0.03197838, -0.01971585,
       -0.01420012,  0.00837301,  0.07664964,  0.02595568,  0.01369434,
        0.04774027,  0.03335406, -0.05015922, -0.01659513, -0.04133891,
        0.02143592,  0.01798619, -0.02611958,  0.05418615,  0.00848071])

## Scale

In [41]:
np.sum(MFact.Weights[i]==1)

102

In [42]:
np.sum(MFact.Idx[i,:])

102

In [43]:
len(Item_Users[i])

102

array([-0.02109893,  0.04068626,  0.07676537,  0.04703615, -0.02490458,
        0.0152375 ,  0.0334261 ,  0.00705645,  0.02915467,  0.06518816,
        0.03127026, -0.01469543,  0.05393025,  0.00544302, -0.05670821,
       -0.02360427, -0.00050787, -0.03409838, -0.06166553, -0.01579171,
       -0.02184748,  0.02948321,  0.01342341,  0.04280662,  0.01198426,
        0.02121263, -0.0049326 , -0.02598882, -0.02511188, -0.0352744 ,
       -0.02801222,  0.00211184, -0.01637119, -0.04229047,  0.03209561,
       -0.00439945,  0.00230341,  0.07112424, -0.01538775, -0.02387161,
       -0.0197786 , -0.01241253,  0.01236529, -0.01592524,  0.00027657,
        0.02109335,  0.0241652 ,  0.01671699, -0.00186549, -0.03361442])

In [80]:
Wi.shape

(1, 102)

In [44]:
W[i, i_rated]

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.])

In [47]:
U.shape

(5436, 50)

TypeError: unhashable type: 'list'